In [208]:
import os
import io
import struct
from typing import List, Tuple, Optional
from enum import Enum
import pandas as pd

from blowfish import Blowfish

In [209]:
class PackEntryType(Enum):
  Nop = 0
  Folder = 1
  File = 2

In [210]:
def generate_final_blowfish_key(password: str, salt: bytes) -> bytes:
    """
    Reproduces the 'GenerateFinalBlowfishKey' logic in C#.
    """
    # 1) Limit key length to max of 56
    plain_key_length = min(len(password), 56)

    # 2) Convert password to ASCII bytes
    a_key = password.encode("ascii")

    # 3) Create a 56-byte base key buffer
    b_key = bytearray(56)
    
    # Copy salt into b_key
    # Equivalent to `Array.ConstrainedCopy(salt, 0, bKey, 0, salt.Length)`
    b_key[:len(salt)] = salt

    # 4) Generate the final blowfish key by XOR-ing
    #    the ASCII password bytes with the corresponding part of b_key.
    bf_key = bytearray(plain_key_length)
    for x in range(plain_key_length):
        bf_key[x] = a_key[x] ^ b_key[x]

    return bytes(bf_key)

In [211]:
def read_string_with_length(stream, byte_count: int) -> str:
    """
    Read 'byte_count' bytes, then decode using self._encoding,
    stopping at the first 0 (null terminator) if present.
    """
    buffer = stream.read(byte_count)

    # Find null terminator offset (if any)
    terminator_offset = byte_count
    for i in range(byte_count):
        if buffer[i] == 0:
            terminator_offset = i
            break

    return buffer[:terminator_offset].decode('ascii', errors='replace')

In [212]:
# data_pk2_path = os.path.join(os.getcwd(), 'data', 'Data.pk2')
media_pk2_path = os.path.join(os.getcwd(), 'data', 'Media.pk2')
# print(data_pk2_path)
print(media_pk2_path)

/Users/hodung/Desktop/Workspace/silkroad/pk2-extractor/data/Media.pk2


In [213]:
pk2key = '169841'
salt = [0x03, 0xF8, 0xE4, 0x44, 0x88, 0x99, 0x3F, 0x64, 0xFE, 0x35]

In [214]:
key = generate_final_blowfish_key(pk2key, bytes(salt))
key

b'2\xce\xdd|\xbc\xa8'

In [215]:
blowfish = Blowfish()
blowfish.Initialize(key)

In [216]:
_media_file_stream = open(media_pk2_path, 'rb')

In [217]:
signature = _media_file_stream.read(30)
version =  struct.unpack('<i', _media_file_stream.read(4))[0]
encrypted = _media_file_stream.read(1)
encryption_checksum = _media_file_stream.read(16)
payload = _media_file_stream.read(205)


In [218]:
Header = {
    'signature': signature,
    'version': version,
    'encrypted': encrypted,
    'encryption_checksum': encryption_checksum,
    'payload': payload
}

In [219]:
blowfish_checksum_decoded = "Joymax Pak File"

In [220]:
if blowfish and encrypted == b'\x01':
    temp_checksum = blowfish.Encode(blowfish_checksum_decoded.encode('ascii'))
    if temp_checksum is None or temp_checksum[0] != encryption_checksum[0] or temp_checksum[1] != encryption_checksum[1] or temp_checksum[2] != encryption_checksum[2]:
        raise Exception('Failed to open JoymaxPackFile: The password or salt is wrong.')

In [221]:
def read_block_at(position: int):
    _media_file_stream.seek(position, io.SEEK_SET)
    buffer = _media_file_stream.read(128 * 20)
    if blowfish is not None:
      entry_buffer = io.BytesIO(blowfish.Decode(buffer))
    else:
      entry_buffer = io.BytesIO(buffer)

    entries = []
    for _ in range(20):
      entry = {
        "Type": PackEntryType(entry_buffer.read(1)[0]),
        "Name": read_string_with_length(entry_buffer, 89).rstrip('\0'),
        "CreationTime": struct.unpack('<q', entry_buffer.read(8))[0],
        "ModifyTime": struct.unpack('<q', entry_buffer.read(8))[0],
        "DataPosition": struct.unpack('<q', entry_buffer.read(8))[0],
        "DataSize": struct.unpack('<i', entry_buffer.read(4))[0],
        "NextBlock": struct.unpack('<q', entry_buffer.read(8))[0],
        "Payload": entry_buffer.read(2)
      }
      entries.append(entry)
      # print(entry)

    return {
      "Position": position,
      "Entries": entries
    }


def read_blocks_at(position: int):
    result = []

    block = read_block_at(position)
    result.append(block)

    if block["Entries"][19]["NextBlock"] > 0:
        result.extend(read_blocks_at(block["Entries"][19]["NextBlock"]))

    return result

In [222]:
Root = read_blocks_at(256)

In [223]:
blocks_in_memory = {
  "": Root
}

In [224]:
blocks_in_memory

{'': [{'Position': 256,
   'Entries': [{'Type': <PackEntryType.Folder: 1>,
     'Name': '.',
     'CreationTime': 133336205048819447,
     'ModifyTime': 133336205048819447,
     'DataPosition': 256,
     'DataSize': 0,
     'NextBlock': 0,
     'Payload': b'\x00\x00'},
    {'Type': <PackEntryType.Folder: 1>,
     'Name': 'acobject',
     'CreationTime': 133336205048829450,
     'ModifyTime': 133336205048829450,
     'DataPosition': 2816,
     'DataSize': 0,
     'NextBlock': 0,
     'Payload': b'\x00\x00'},
    {'Type': <PackEntryType.Folder: 1>,
     'Name': 'config',
     'CreationTime': 133336205049179438,
     'ModifyTime': 133336205049179438,
     'DataPosition': 35136,
     'DataSize': 0,
     'NextBlock': 0,
     'Payload': b'\x00\x00'},
    {'Type': <PackEntryType.Folder: 1>,
     'Name': 'Effect',
     'CreationTime': 133336205049679437,
     'ModifyTime': 133336205049679437,
     'DataPosition': 41896,
     'DataSize': 0,
     'NextBlock': 0,
     'Payload': b'\x00\x00'},
   

## DIVISIONINFO.TXT

In [225]:
division_info_txt = "DIVISIONINFO.TXT"

In [226]:
root = blocks_in_memory[""]
entry = None
for entries in root:
  entry = next((x for x in entries["Entries"] if x["Name"] == division_info_txt), None)
  if entry:
    break

entry

{'Type': <PackEntryType.File: 2>,
 'Name': 'DIVISIONINFO.TXT',
 'CreationTime': 133336210349991177,
 'ModifyTime': 133336210349991177,
 'DataPosition': 1017319424,
 'DataSize': 57,
 'NextBlock': 0,
 'Payload': b'\x00\x00'}

In [227]:
_media_file_stream.seek(entry["DataPosition"], io.SEEK_SET)
buffer = io.BytesIO(_media_file_stream.read(entry["DataSize"]))

Locale = buffer.read(1)[0]
division_count = buffer.read(1)[0]

for i in range(division_count):
  length = struct.unpack('<i', buffer.read(4))[0]
  division_name = buffer.read(length).decode('utf-8')
  print(division_name)
  buffer.read(1) # Skip null terminator
  gateway_count = buffer.read(1)[0]
  for j in range(gateway_count):
    length = struct.unpack('<i', buffer.read(4))[0]
    gateway_name = buffer.read(length).decode('utf-8')
    print(gateway_name)
    buffer.read(1)

DIV01
gw1sro.vtcgame.vn
gw2sro.vtcgame.vn


## GATEPORT.TXT

In [228]:
gate_port_txt = "GATEPORT.TXT"

In [229]:
root = blocks_in_memory[""]
entry = None
for entries in root:
  entry = next((x for x in entries["Entries"] if x["Name"] == gate_port_txt), None)
  if entry:
    break

entry

{'Type': <PackEntryType.File: 2>,
 'Name': 'GATEPORT.TXT',
 'CreationTime': 133336210351001190,
 'ModifyTime': 133336210351001190,
 'DataPosition': 1017323065,
 'DataSize': 8,
 'NextBlock': 0,
 'Payload': b'\x00\x00'}

In [230]:
_media_file_stream.seek(entry["DataPosition"], io.SEEK_SET)
buffer = _media_file_stream.read(entry["DataSize"])

In [231]:
port = int(buffer.decode('ascii').rstrip('\0'))
port

15779

## SV.T

In [232]:
sv_t = "SV.T"

In [233]:
root = blocks_in_memory[""]
entry = None
for entries in root:
  entry = next((x for x in entries["Entries"] if x["Name"] == sv_t), None)
  if entry:
    break

entry

{'Type': <PackEntryType.File: 2>,
 'Name': 'SV.T',
 'CreationTime': 133795823380867913,
 'ModifyTime': 133795823380867913,
 'DataPosition': 1017319481,
 'DataSize': 1024,
 'NextBlock': 1017320505,
 'Payload': b'\x00\x00'}

In [234]:
_media_file_stream.seek(entry["DataPosition"], io.SEEK_SET)
buffer = io.BytesIO(_media_file_stream.read(entry["DataSize"]))

In [235]:
version_buffer_length = struct.unpack('<i', buffer.read(4))[0]
version_buffer = buffer.read(version_buffer_length)

In [236]:
key = "SILKROADVERSION"
key_buffer = key.encode('ascii')[:8]
sv_blowfish = Blowfish()
sv_blowfish.Initialize(key_buffer)

In [237]:
decoded_version_buffer = sv_blowfish.Decode(version_buffer)[:4]
version = int(decoded_version_buffer.decode('ascii').rstrip('\0'))
version

15

In [238]:
def get_entry_buffer(file_path: str) -> Optional[dict]:
  parent_folder_path = os.path.dirname(file_path)
  file_name = os.path.basename(file_path)
  if parent_folder_path not in blocks_in_memory:
    paths = parent_folder_path.split(os.path.sep)
    blocks = blocks_in_memory[""]
    current_path = ""

    for sub_folder_name in paths:
        for block in blocks:
            entries = block["Entries"]
            entry = next((e for e in entries if e["Name"] == sub_folder_name and e["Type"] == PackEntryType.Folder), None)
            if entry is None:
                continue
            
            current_path = os.path.join(current_path, entry["Name"])
            if current_path in blocks_in_memory:
                blocks = blocks_in_memory[current_path]
                break
            
            blocks = read_blocks_at(entry["DataPosition"])
            blocks_in_memory[current_path] = blocks
            break
        
  root = blocks_in_memory[parent_folder_path]
  entry = None
  for entries in root:
    entry = next((x for x in entries["Entries"] if x["Name"] == file_name.lower()), None)
    if entry:
      break

  _media_file_stream.seek(entry["DataPosition"], io.SEEK_SET)
  # _media_file_stream.read(entry["DataSize"])
  buffer = io.BytesIO(_media_file_stream.read(entry["DataSize"])) 

  return buffer
   

def get_lines(file_path: str) -> List[Tuple[str, int]]:

  buffer = get_entry_buffer(file_path)

  text = buffer.read().decode('utf-16', errors="replace")
  lines = text.split('\r\n')
  return lines  

def convert_string(value):
    try:
        # First, try to convert to an integer
        return int(value)
    except ValueError:
        try:
            # If it fails, try to convert to a float
            return float(value)
        except ValueError:
            # If both conversions fail, return the original string
            return value


In [239]:
text_data_directory = os.path.join('server_dep', 'silkroad', 'textdata')
text_data_directory

'server_dep/silkroad/textdata'

In [240]:
character_data_list_file = os.path.join(text_data_directory, 'CharacterData.txt')
character_data_list_file

'server_dep/silkroad/textdata/CharacterData.txt'

In [241]:
character_data_files = get_lines(character_data_list_file)
character_data_files

['CharacterData_10000.txt',
 'CharacterData_10100.txt',
 'CharacterData_10200.txt',
 'CharacterData_10300.txt',
 'CharacterData_14800.txt',
 'CharacterData_14900.txt',
 'CharacterData_15000.txt',
 'CharacterData_15100.txt',
 'CharacterData_15200.txt',
 'CharacterData_15300.txt',
 'CharacterData_15400.txt',
 'CharacterData_15500.txt',
 'CharacterData_15600.txt',
 'CharacterData_15700.txt',
 'CharacterData_15800.txt',
 'CharacterData_15900.txt',
 'CharacterData_16000.txt',
 'CharacterData_16100.txt',
 'CharacterData_16200.txt',
 'CharacterData_16300.txt',
 'CharacterData_16400.txt',
 'CharacterData_16500.txt',
 'CharacterData_16600.txt',
 'CharacterData_16700.txt',
 'CharacterData_16800.txt',
 'CharacterData_16900.txt',
 'CharacterData_17000.txt',
 'CharacterData_17100.txt',
 'CharacterData_17200.txt',
 'CharacterData_17300.txt',
 'CharacterData_17400.txt',
 'CharacterData_17500.txt',
 'CharacterData_17600.txt',
 'CharacterData_17700.txt',
 'CharacterData_17800.txt',
 'CharacterData_1790

In [242]:
df_character_data = pd.DataFrame(columns=range(104))

for character_data_file in character_data_files:
  file_name = os.path.join(text_data_directory, character_data_file)
  lines = get_lines(file_name)
  for idx, line in enumerate(lines):
    values = line.split('\t')
    if len(values) < 104:
      continue
    df_character_data.loc[len(df_character_data)] = values
    


In [243]:
df_character_data

,0,1,2,3,4,5,6,7,8,9,...,94,95,96,97,98,99,100,101,102,103
0,1,9900,COS_GUILD_CH_SOLDIER5_059,???? lv 59,COS_GUILD_CH_SOLDIER5_001,SN_COS_GUILD_CH_SOLDIER5,xxx,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,1,9901,COS_GUILD_CH_SOLDIER5_060,???? lv 60,COS_GUILD_CH_SOLDIER5_001,SN_COS_GUILD_CH_SOLDIER5,xxx,0,1,1,...,0,0,0,0,0,0,0,0,0,0
2,1,9902,COS_GUILD_CH_SOLDIER5_061,???? lv 61,COS_GUILD_CH_SOLDIER5_001,SN_COS_GUILD_CH_SOLDIER5,xxx,0,1,1,...,0,0,0,0,0,0,0,0,0,0
3,1,9903,COS_GUILD_CH_SOLDIER5_062,???? lv 62,COS_GUILD_CH_SOLDIER5_001,SN_COS_GUILD_CH_SOLDIER5,xxx,0,1,1,...,0,0,0,0,0,0,0,0,0,0
4,1,9904,COS_GUILD_CH_SOLDIER5_063,???? lv 63,COS_GUILD_CH_SOLDIER5_001,SN_COS_GUILD_CH_SOLDIER5,xxx,0,1,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14626,1,9895,COS_GUILD_CH_SOLDIER5_054,???? lv 54,COS_GUILD_CH_SOLDIER5_001,SN_COS_GUILD_CH_SOLDIER5,xxx,0,1,1,...,0,0,0,0,0,0,0,0,0,0
14627,1,9896,COS_GUILD_CH_SOLDIER5_055,???? lv 55,COS_GUILD_CH_SOLDIER5_001,SN_COS_GUILD_CH_SOLDIER5,xxx,0,1,1,...,0,0,0,0,0,0,0,0,0,0
14628,1,9897,COS_GUILD_CH_SOLDIER5_056,???? lv 56,COS_GUILD_CH_SOLDIER5_001,SN_COS_GUILD_CH_SOLDIER5,xxx,0,1,1,...,0,0,0,0,0,0,0,0,0,0
14629,1,9898,COS_GUILD_CH_SOLDIER5_057,???? lv 57,COS_GUILD_CH_SOLDIER5_001,SN_COS_GUILD_CH_SOLDIER5,xxx,0,1,1,...,0,0,0,0,0,0,0,0,0,0
